<a href="https://colab.research.google.com/github/ADMoreau/MAMLGAN/blob/master/mamlgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import cv2
import torch
import requests
import numpy as np
import gc
import pandas as pd
from pycocotools.coco import COCO
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
from PIL import Image
from IPython.display import display, HTML, clear_output
from ipywidgets import widgets, Layout
from io import BytesIO
import argparse
import os
import numpy as np
import math
import itertools
import datetime
import time
import sys
from tqdm import trange

from torch.distributions import Normal, Categorical
import torchvision
from torchvision import datasets, models, transforms
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch
from copy import deepcopy
import scipy
import scipy.misc
from torch.autograd import Variable
from torchvision.models.utils import load_state_dict_from_url


cuda = torch.cuda.is_available()

if cuda:
  device = torch.device("cuda:0")

In [0]:
!pip install --upgrade efficientnet-pytorch

Requirement already up-to-date: efficientnet-pytorch in /usr/local/lib/python3.6/dist-packages (0.6.1)


In [0]:
from efficientnet_pytorch.utils import (
    round_filters,
    round_repeats,
    drop_connect,
    get_same_padding_conv2d,
    get_model_params,
    efficientnet_params,
    load_pretrained_weights,
    Swish,
    MemoryEfficientSwish,
)

class MBConvBlock(nn.Module):
    """
    Mobile Inverted Residual Bottleneck Block
    Args:
        block_args (namedtuple): BlockArgs, see above
        global_params (namedtuple): GlobalParam, see above
    Attributes:
        has_se (bool): Whether the block contains a Squeeze and Excitation layer.
    """

    def __init__(self, block_args, global_params):
        super().__init__()
        self._block_args = block_args
        self._bn_mom = 1 - global_params.batch_norm_momentum
        self._bn_eps = global_params.batch_norm_epsilon
        self.has_se = (self._block_args.se_ratio is not None) and (0 < self._block_args.se_ratio <= 1)
        self.id_skip = block_args.id_skip  # skip connection and drop connect

        # Get static or dynamic convolution depending on image size
        Conv2d = get_same_padding_conv2d(image_size=global_params.image_size)

        # Expansion phase
        inp = self._block_args.input_filters  # number of input channels
        oup = self._block_args.input_filters * self._block_args.expand_ratio  # number of output channels
        if self._block_args.expand_ratio != 1:
            self._expand_conv = Conv2d(in_channels=inp, out_channels=oup, kernel_size=1, bias=False)
            self._bn0 = nn.BatchNorm2d(num_features=oup, momentum=self._bn_mom, eps=self._bn_eps)

        # Depthwise convolution phase
        k = self._block_args.kernel_size
        s = self._block_args.stride
        self._depthwise_conv = Conv2d(
            in_channels=oup, out_channels=oup, groups=oup,  # groups makes it depthwise
            kernel_size=k, stride=s, bias=False)
        self._bn1 = nn.BatchNorm2d(num_features=oup, momentum=self._bn_mom, eps=self._bn_eps)

        # Squeeze and Excitation layer, if desired
        if self.has_se:
            num_squeezed_channels = max(1, int(self._block_args.input_filters * self._block_args.se_ratio))
            self._se_reduce = Conv2d(in_channels=oup, out_channels=num_squeezed_channels, kernel_size=1)
            self._se_expand = Conv2d(in_channels=num_squeezed_channels, out_channels=oup, kernel_size=1)

        # Output phase
        final_oup = self._block_args.output_filters
        self._project_conv = Conv2d(in_channels=oup, out_channels=final_oup, kernel_size=1, bias=False)
        self._bn2 = nn.BatchNorm2d(num_features=final_oup, momentum=self._bn_mom, eps=self._bn_eps)
        self._swish = MemoryEfficientSwish()

    def forward(self, inputs, drop_connect_rate=None):
        """
        :param inputs: input tensor
        :param drop_connect_rate: drop connect rate (float, between 0 and 1)
        :return: output of block
        """

        # Expansion and Depthwise Convolution
        x = inputs
        if self._block_args.expand_ratio != 1:
            x = self._swish(self._bn0(self._expand_conv(inputs)))
        x = self._swish(self._bn1(self._depthwise_conv(x)))

        # Squeeze and Excitation
        if self.has_se:
            x_squeezed = F.adaptive_avg_pool2d(x, 1)
            x_squeezed = self._se_expand(self._swish(self._se_reduce(x_squeezed)))
            x = torch.sigmoid(x_squeezed) * x

        x = self._bn2(self._project_conv(x))

        # Skip connection and drop connect
        input_filters, output_filters = self._block_args.input_filters, self._block_args.output_filters
        if self.id_skip and self._block_args.stride == 1 and input_filters == output_filters:
            if drop_connect_rate:
                x = drop_connect(x, p=drop_connect_rate, training=self.training)
            x = x + inputs  # skip connection
        return x

    def set_swish(self, memory_efficient=True):
        """Sets swish function as memory efficient (for training) or standard (for export)"""
        self._swish = MemoryEfficientSwish() if memory_efficient else Swish()


class EfficientNet(nn.Module):
    """
    An EfficientNet model. Most easily loaded with the .from_name or .from_pretrained methods
    Args:
        blocks_args (list): A list of BlockArgs to construct blocks
        global_params (namedtuple): A set of GlobalParams shared between blocks
    Example:
        model = EfficientNet.from_pretrained('efficientnet-b0')
    """

    def __init__(self, blocks_args=None, global_params=None):
        super().__init__()
        assert isinstance(blocks_args, list), 'blocks_args should be a list'
        assert len(blocks_args) > 0, 'block args must be greater than 0'
        self._global_params = global_params
        self._blocks_args = blocks_args

        # Get static or dynamic convolution depending on image size
        Conv2d = get_same_padding_conv2d(image_size=global_params.image_size)

        # Batch norm parameters
        bn_mom = 1 - self._global_params.batch_norm_momentum
        bn_eps = self._global_params.batch_norm_epsilon

        # Stem
        in_channels = 3  # rgb
        out_channels = round_filters(32, self._global_params)  # number of output channels
        self._conv_stem = Conv2d(in_channels, out_channels, kernel_size=3, stride=2, bias=False)
        self._bn0 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)

        # Build blocks
        self._blocks = nn.ModuleList([])
        for block_args in self._blocks_args:

            # Update block input and output filters based on depth multiplier.
            block_args = block_args._replace(
                input_filters=round_filters(block_args.input_filters, self._global_params),
                output_filters=round_filters(block_args.output_filters, self._global_params),
                num_repeat=round_repeats(block_args.num_repeat, self._global_params)
            )

            # The first block needs to take care of stride and filter size increase.
            self._blocks.append(MBConvBlock(block_args, self._global_params))
            if block_args.num_repeat > 1:
                block_args = block_args._replace(input_filters=block_args.output_filters, stride=1)
            for _ in range(block_args.num_repeat - 1):
                self._blocks.append(MBConvBlock(block_args, self._global_params))

        # Head
        in_channels = block_args.output_filters  # output of final block
        out_channels = round_filters(1280, self._global_params)
        self._conv_head = Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self._bn1 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)

        # Final linear layer
        self._avg_pooling = nn.AdaptiveAvgPool2d(1)
        self._dropout = nn.Dropout(self._global_params.dropout_rate)
        self._fc = nn.Linear(out_channels, self._global_params.num_classes)
        self._swish = MemoryEfficientSwish()

    def set_swish(self, memory_efficient=True):
        """Sets swish function as memory efficient (for training) or standard (for export)"""
        self._swish = MemoryEfficientSwish() if memory_efficient else Swish()
        for block in self._blocks:
            block.set_swish(memory_efficient)


    def extract_features(self, inputs):
        """ Returns output of the final convolution layer """

        # Stem
        x = self._swish(self._bn0(self._conv_stem(inputs)))

        # Blocks
        for idx, block in enumerate(self._blocks):
            drop_connect_rate = self._global_params.drop_connect_rate
            if drop_connect_rate:
                drop_connect_rate *= float(idx) / len(self._blocks)
            x = block(x, drop_connect_rate=drop_connect_rate)

        # Head
        x = self._swish(self._bn1(self._conv_head(x)))

        return x

    def classify(self, inputs):
        bs = inputs.size(0)
        # Pooling and final linear layer
        x = self._avg_pooling(inputs)
        x = x.view(bs, -1)
        x = self._dropout(x)
        x = self._fc(x)
        return x

    def forward(self, inputs):
        """ Calls extract_features to extract features, applies final linear layer, and returns logits. """
        # Convolution layers
        x = self.extract_features(inputs)
        x = self.classify(x)
        return x
        

    @classmethod
    def from_name(cls, model_name, override_params=None):
        cls._check_model_name_is_valid(model_name)
        blocks_args, global_params = get_model_params(model_name, override_params)
        return cls(blocks_args, global_params)

    @classmethod
    def from_pretrained(cls, model_name, advprop=False, num_classes=1000, in_channels=3):
        model = cls.from_name(model_name, override_params={'num_classes': num_classes})
        load_pretrained_weights(model, model_name, load_fc=(num_classes == 1000), advprop=advprop)
        if in_channels != 3:
            Conv2d = get_same_padding_conv2d(image_size = model._global_params.image_size)
            out_channels = round_filters(32, model._global_params)
            model._conv_stem = Conv2d(in_channels, out_channels, kernel_size=3, stride=2, bias=False)
        return model
    
    @classmethod
    def get_image_size(cls, model_name):
        cls._check_model_name_is_valid(model_name)
        _, _, res, _ = efficientnet_params(model_name)
        return res

    @classmethod
    def _check_model_name_is_valid(cls, model_name):
        """ Validates model name. """ 
        valid_models = ['efficientnet-b'+str(i) for i in range(9)]
        if model_name not in valid_models:
            raise ValueError('model_name should be one of: ' + ', '.join(valid_models))

In [0]:
efficientnet_transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

efficientnet = EfficientNet.from_name('efficientnet-b5') 
out_channels = round_filters(1280, efficientnet._global_params)
efficientnet._fc = nn.Linear(out_channels, 10)
efficientnet = efficientnet.to(device)
efficientnet.load_state_dict(torch.load('/content/drive/My Drive/cifar10.pth'))
efficientnet.eval()

MUNIT Constructor

https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/munit

In [0]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class LambdaLR:
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert (n_epochs - decay_start_epoch) > 0, "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)


class Encoder(nn.Module):
    def __init__(self, in_channels=3, dim=64, n_residual=3, n_downsample=2, style_dim=8):
        super(Encoder, self).__init__()
        self.content_encoder = ContentEncoder(in_channels, dim, n_residual, n_downsample)
        self.style_encoder = StyleEncoder(in_channels, dim, n_downsample, style_dim)

    def forward(self, x):
        content_code = self.content_encoder(x)
        style_code = self.style_encoder(x)
        return content_code, style_code


class Decoder(nn.Module):
    def __init__(self, out_channels=3, dim=64, n_residual=3, n_upsample=2, style_dim=8):
        super(Decoder, self).__init__()

        layers = []
        dim = dim * 2 ** n_upsample
        # Residual blocks
        for _ in range(n_residual):
            layers += [ResidualBlock(dim, norm="adain")]

        # Upsampling
        for _ in range(n_upsample):
            layers += [
                nn.Upsample(scale_factor=2),
                nn.Conv2d(dim, dim // 2, 5, stride=1, padding=2),
                LayerNorm(dim // 2),
                nn.ReLU(inplace=True),
            ]
            dim = dim // 2

        # Output layer
        layers += [nn.ReflectionPad2d(3), nn.Conv2d(dim, out_channels, 7), nn.Tanh()]

        self.model = nn.Sequential(*layers)

        # Initiate mlp (predicts AdaIN parameters)
        num_adain_params = self.get_num_adain_params()
        self.mlp = MLP(style_dim, num_adain_params)

    def get_num_adain_params(self):
        """Return the number of AdaIN parameters needed by the model"""
        num_adain_params = 0
        for m in self.modules():
            if m.__class__.__name__ == "AdaptiveInstanceNorm2d":
                num_adain_params += 2 * m.num_features
        return num_adain_params

    def assign_adain_params(self, adain_params):
        """Assign the adain_params to the AdaIN layers in model"""
        for m in self.modules():
            if m.__class__.__name__ == "AdaptiveInstanceNorm2d":
                # Extract mean and std predictions
                mean = adain_params[:, : m.num_features]
                std = adain_params[:, m.num_features : 2 * m.num_features]
                # Update bias and weight
                m.bias = mean.contiguous().view(-1)
                m.weight = std.contiguous().view(-1)
                # Move pointer
                if adain_params.size(1) > 2 * m.num_features:
                    adain_params = adain_params[:, 2 * m.num_features :]

    def forward(self, content_code, style_code):
        # Update AdaIN parameters by MLP prediction based off style code
        self.assign_adain_params(self.mlp(style_code))
        img = self.model(content_code)
        return img


class ContentEncoder(nn.Module):
    def __init__(self, in_channels=3, dim=64, n_residual=3, n_downsample=2):
        super(ContentEncoder, self).__init__()

        # Initial convolution block
        layers = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(in_channels, dim, 7),
            nn.InstanceNorm2d(dim),
            nn.ReLU(inplace=True),
        ]

        # Downsampling
        for _ in range(n_downsample):
            layers += [
                nn.Conv2d(dim, dim * 2, 4, stride=2, padding=1),
                nn.InstanceNorm2d(dim * 2),
                nn.ReLU(inplace=True),
            ]
            dim *= 2

        # Residual blocks
        for _ in range(n_residual):
            layers += [ResidualBlock(dim, norm="in")]

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


class StyleEncoder(nn.Module):
    def __init__(self, in_channels=3, dim=64, n_downsample=2, style_dim=8):
        super(StyleEncoder, self).__init__()

        # Initial conv block
        layers = [nn.ReflectionPad2d(3), nn.Conv2d(in_channels, dim, 7), nn.ReLU(inplace=True)]

        # Downsampling
        for _ in range(2):
            layers += [nn.Conv2d(dim, dim * 2, 4, stride=2, padding=1), nn.ReLU(inplace=True)]
            dim *= 2

        # Downsampling with constant depth
        for _ in range(n_downsample - 2):
            layers += [nn.Conv2d(dim, dim, 4, stride=2, padding=1), nn.ReLU(inplace=True)]

        # Average pool and output layer
        layers += [nn.AdaptiveAvgPool2d(1), nn.Conv2d(dim, style_dim, 1, 1, 0)]

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, dim=256, n_blk=3, activ="relu"):
        super(MLP, self).__init__()
        layers = [nn.Linear(input_dim, dim), nn.ReLU(inplace=True)]
        for _ in range(n_blk - 2):
            layers += [nn.Linear(dim, dim), nn.ReLU(inplace=True)]
        layers += [nn.Linear(dim, output_dim)]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x.view(x.size(0), -1))


class MultiDiscriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(MultiDiscriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalize=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        # Extracts three discriminator models
        self.models = nn.ModuleList()
        for i in range(3):
            self.models.add_module(
                "disc_%d" % i,
                nn.Sequential(
                    *discriminator_block(in_channels, 64, normalize=False),
                    *discriminator_block(64, 128),
                    *discriminator_block(128, 256),
                    *discriminator_block(256, 512),
                    nn.Conv2d(512, 1, 3, padding=1)
                ),
            )

        self.downsample = nn.AvgPool2d(in_channels, stride=2, padding=[1, 1], count_include_pad=False)

    def compute_loss(self, x, gt):
        """Computes the MSE between model output and scalar gt"""
        loss = sum([torch.mean((out - gt) ** 2) for out in self.forward(x)])
        return loss

    def forward(self, x):
        outputs = []
        for m in self.models:
            outputs.append(m(x))
            x = self.downsample(x)
        return outputs


class ResidualBlock(nn.Module):
    def __init__(self, features, norm="in"):
        super(ResidualBlock, self).__init__()

        norm_layer = AdaptiveInstanceNorm2d if norm == "adain" else nn.InstanceNorm2d

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(features, features, 3),
            norm_layer(features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(features, features, 3),
            norm_layer(features),
        )

    def forward(self, x):
        return x + self.block(x)


class AdaptiveInstanceNorm2d(nn.Module):
    """Reference: https://github.com/NVlabs/MUNIT/blob/master/networks.py"""

    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(AdaptiveInstanceNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        # weight and bias are dynamically assigned
        self.weight = None
        self.bias = None
        # just dummy buffers, not used
        self.register_buffer("running_mean", torch.zeros(num_features))
        self.register_buffer("running_var", torch.ones(num_features))

    def forward(self, x):
        assert (
            self.weight is not None and self.bias is not None
        ), "Please assign weight and bias before calling AdaIN!"
        b, c, h, w = x.size()
        running_mean = self.running_mean.repeat(b)
        running_var = self.running_var.repeat(b)

        # Apply instance norm
        x_reshaped = x.contiguous().view(1, b * c, h, w)

        out = F.batch_norm(
            x_reshaped, running_mean, running_var, self.weight, self.bias, True, self.momentum, self.eps
        )

        return out.view(b, c, h, w)

    def __repr__(self):
        return self.__class__.__name__ + "(" + str(self.num_features) + ")"


class LayerNorm(nn.Module):
    def __init__(self, num_features, eps=1e-5, affine=True):
        super(LayerNorm, self).__init__()
        self.num_features = num_features
        self.affine = affine
        self.eps = eps

        if self.affine:
            self.gamma = nn.Parameter(torch.Tensor(num_features).uniform_())
            self.beta = nn.Parameter(torch.zeros(num_features))

    def forward(self, x):
        shape = [-1] + [1] * (x.dim() - 1)
        mean = x.view(x.size(0), -1).mean(1).view(*shape)
        std = x.view(x.size(0), -1).std(1).view(*shape)
        x = (x - mean) / (std + self.eps)

        if self.affine:
            shape = [1, -1] + [1] * (x.dim() - 2)
            x = x * self.gamma.view(*shape) + self.beta.view(*shape)
        return x

In [0]:
def discount_rewards(r, gamma):
  discounted_r = torch.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size(-1))):
      running_add = running_add * gamma + r[t]
      discounted_r[t] = running_add
  return discounted_r

class StylePolicy(nn.Module):
  def __init__(self, latent_dim=512, img_size = 64, window_size = 16):
    super(StylePolicy, self).__init__()
    self.gamma = 0.99
    self.action_probs = []
    self.saved_log_probs = []
    self.rewards = []

    self.fc = nn.Linear(8192, 1024)
    self.ReLU1_1 = nn.ReLU(inplace=True)

    self.fc_2 = nn.Linear(1044, 512)
    self.ReLU1_2 = nn.ReLU(inplace=True)

    self.lfc = nn.Linear(latent_dim, 256)
    self.lat = nn.Linear(256, img_size - window_size)
    self.lon = nn.Linear(256, img_size - window_size)

    self.fc_a = nn.Linear(latent_dim, 256)
    self.mean_a = nn.Linear(256, 1)
    self.sigma_a = nn.Linear(256, 1) 

    self.fc_b = nn.Linear(latent_dim, 256)
    self.mean_b = nn.Linear(256, 1)
    self.sigma_b = nn.Linear(256, 1) 

    self.fc_c = nn.Linear(latent_dim, 256)
    self.mean_c = nn.Linear(256, 1)
    self.sigma_c = nn.Linear(256, 1) 

    self.fc_d = nn.Linear(latent_dim, 256)
    self.mean_d = nn.Linear(256, 1)
    self.sigma_d = nn.Linear(256, 1) 

    self.fc_e = nn.Linear(latent_dim, 256)
    self.mean_e = nn.Linear(256, 1)
    self.sigma_e = nn.Linear(256, 1) 

    self.fc_f = nn.Linear(latent_dim, 256)
    self.mean_f = nn.Linear(256, 1)
    self.sigma_f = nn.Linear(256, 1) 

    self.fc_g = nn.Linear(latent_dim, 256)
    self.mean_g = nn.Linear(256, 1)
    self.sigma_g = nn.Linear(256, 1) 

    self.fc_h = nn.Linear(latent_dim, 256)
    self.mean_h = nn.Linear(256, 1)
    self.sigma_h = nn.Linear(256, 1) 

  def forward(self, efficientnet_encoding, efficientnet_output, goal_vector):
    r = torch.flatten(efficientnet_encoding, 1)
    r = self.fc(r)
    r = self.ReLU1_1(r)

    z = torch.cat((r, efficientnet_output, goal_vector), 1)
    z = self.ReLU1_2(self.fc_2(z))

    # Decode the hidden state of the last time step
    lz = nn.ReLU()(self.lfc(z))
    lat = Categorical(nn.Softmax(dim=1)(self.lat(lz)))
    lon = Categorical(nn.Softmax(dim=1)(self.lon(lz)))

    z_a = nn.ReLU()(self.fc_a(z))
    mu_a = self.mean_a(z_a)
    sig_a = nn.Softplus()(self.sigma_a(z_a)) + 1e-5
    style_a = Normal(mu_a, sig_a)

    z_b = nn.ReLU()(self.fc_b(z))
    mu_b = self.mean_b(z_b)
    sig_b = nn.Softplus()(self.sigma_b(z_b)) + 1e-5
    style_b = Normal(mu_b, sig_b)

    z_c = nn.ReLU()(self.fc_c(z))
    mu_c = self.mean_c(z_c)
    sig_c = nn.Softplus()(self.sigma_c(z_c)) + 1e-5
    style_c = Normal(mu_c, sig_c)

    z_d = nn.ReLU()(self.fc_d(z))
    mu_d = self.mean_d(z_d)
    sig_d = nn.Softplus()(self.sigma_d(z_d)) + 1e-5
    style_d = Normal(mu_d, sig_d)

    z_e = nn.ReLU()(self.fc_e(z))
    mu_e = self.mean_e(z_e)
    sig_e = nn.Softplus()(self.sigma_e(z_e)) + 1e-5
    style_e = Normal(mu_e, sig_e)

    z_f = nn.ReLU()(self.fc_f(z))
    mu_f = self.mean_f(z_f)
    sig_f = nn.Softplus()(self.sigma_f(z_f)) + 1e-5
    style_f = Normal(mu_f, sig_f)

    z_g = nn.ReLU()(self.fc_g(z))
    mu_g = self.mean_g(z_g)
    sig_g = nn.Softplus()(self.sigma_g(z_g)) + 1e-5
    style_g = Normal(mu_g, sig_g)

    z_h = nn.ReLU()(self.fc_h(z))
    mu_h = self.mean_h(z_h)
    sig_h = nn.Softplus()(self.sigma_h(z_h)) + 1e-5
    style_h = Normal(mu_h, sig_h)
    
    return lat, lon, style_a, style_b, style_c, style_d, style_e, style_f, style_g, style_h

  def get_action(self, efficientnet_encoding, efficientnet_output, goal_vector):
    lat, lon, style_a, style_b, style_c, style_d, style_e, style_f, style_g, style_h = self.forward(efficientnet_encoding, efficientnet_output, goal_vector)
    lat_action = lat.sample() 
    lon_action = lon.sample()
    style_a_action = style_a.sample()
    style_b_action = style_b.sample()
    style_c_action = style_c.sample()
    style_d_action = style_d.sample()
    style_e_action = style_e.sample()
    style_f_action = style_f.sample()
    style_g_action = style_g.sample()
    style_h_action = style_h.sample()
    action_log_prob = lat.log_prob(lat_action) \
                      * lon.log_prob(lon_action) \
                      * style_a.log_prob(style_a_action) \
                      * style_b.log_prob(style_b_action) \
                      * style_c.log_prob(style_c_action) \
                      * style_d.log_prob(style_d_action) \
                      * style_e.log_prob(style_e_action) \
                      * style_f.log_prob(style_f_action) \
                      * style_g.log_prob(style_g_action) \
                      * style_h.log_prob(style_h_action) 
    self.action_probs.append(action_log_prob)
    return lat_action, lon_action, style_a_action, style_b_action, style_c_action, \
            style_d_action, style_e_action, style_f_action, style_g_action, style_h_action

  def episode_finished(self):
    #print("Self.action_probs : {}".format(self.action_probs))
    action_probs = torch.stack(self.action_probs, dim=0) \
            .to(device).squeeze(-1)
    rewards = torch.stack(self.rewards, dim=0).to(device).squeeze(-1)
    self.action_probs, self.rewards = [], []

    G = discount_rewards(rewards, self.gamma)
    #print("G : {}".format(G))
    G = (G - G.mean()) / (G.std() + 1e-9) #normalize discounted rewards
    loss = (-1 * action_probs * G.detach()).sum()
    #print("G : {}, Rewards : {}, Action_probs : {}".format(G, rewards, action_probs))
    sys.stdout.write("\rReinforce Loss : {} Reinforce Mean Reward : {}".format(loss, rewards.detach().mean()))
    sys.stdout.flush()
    return loss

  def store_outcome(self, reward):
    self.rewards.append(torch.Tensor([reward]))

params

In [0]:
dataset_name        = 'cifar10'
epoch               = 215           #epoch to start training from
n_epochs            = 2000         #number of epochs of training
batch_size          = 10         #size of the batches
lr                  = 0.0001      #adam: learning rate
b1                  = 0.5         #adam: decay of first order momentum of gradient
b2                  = 0.999       #adam: decay of first order momentum of gradient
decay_epoch         = 100         #epoch from which to start lr decay
img_height          = 64         #size of image height
img_width           = 64         #size of image width
channels            = 3           #number of image channels
sample_interval     = 400         #interval saving generator samples
n_downsample        = 5           #number downsampling layers in encoder
n_residual          = 2           #number of residual blocks in encoder / decoder
dim                 = 64          #number of filters in first encoder layer
style_dim           = 8        #dimensionality of the style code
reinforce_iterations= 64
inner_loop_batch_size=10          #how many iterations to use for training the discriminator and dec/enc
outerstepsize0       = 0.1 # stepsize of outer optimization, i.e., meta-optimization

In [0]:
from torchvision import datasets, transforms
train_dataset = datasets.CIFAR10(root="./data/",
                                transform=transforms.Compose([
                                    transforms.Resize(64),
                                    transforms.RandomCrop(16),
                                    transforms.Resize(64),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
                                train=True,
                                download=True)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    drop_last=True)

Files already downloaded and verified


In [0]:
# Create sample and checkpoint directories
if not os.path.isdir("/content/drive/My Drive/model"):
  os.makedirs("/content/drive/My Drive/model")
if not os.path.isdir("/content/drive/My Drive/model/images/"):
  os.makedirs("/content/drive/My Drive/model/images/")
if not os.path.isdir("/content/drive/My Drive/model/saved_models/"):
  os.makedirs("/content/drive/My Drive/model/saved_models/")
if not os.path.isdir("/content/drive/My Drive/model/saved_models/cifar10/"):
  os.makedirs("/content/drive/My Drive/model/saved_models/cifar10/")

In [0]:
CrossEntropy_criterion = nn.CrossEntropyLoss()
L1_criterion = torch.nn.L1Loss()

# Initialize encoders, generators and discriminators
StylePolicy = StylePolicy()

Enc = Encoder(dim=dim, n_downsample=n_downsample, n_residual=n_residual, style_dim=style_dim)
Dec = Decoder(dim=dim, n_upsample=n_downsample, n_residual=n_residual, style_dim=style_dim)

D = MultiDiscriminator()

In [0]:
if cuda:
  StylePolicy.to(device)
  Enc = Enc.to(device)
  Dec = Dec.to(device)
  D = D.to(device)

  CrossEntropy_criterion = CrossEntropy_criterion.to(device)
  L1_criterion = L1_criterion.to(device)

if epoch != 0:
# Load pretrained models
  StylePolicy.load_state_dict(torch.load("/content/drive/My Drive/model/saved_models/%s/Policy.pth" % (dataset_name)))
  Enc.load_state_dict(torch.load("/content/drive/My Drive/model/saved_models/%s/Enc.pth" % (dataset_name)))
  Dec.load_state_dict(torch.load("/content/drive/My Drive/model/saved_models/%s/Dec.pth" % (dataset_name)))
  D.load_state_dict(torch.load("/content/drive/My Drive/model/saved_models/%s/D.pth" % (dataset_name)))

else:
  # Initialize weights
  StylePolicy.apply(weights_init_normal)
  Enc.apply(weights_init_normal)
  Dec.apply(weights_init_normal)
  D.apply(weights_init_normal)

# Loss weights
lambda_gan = 5
lambda_style = 10
lambda_cont = 1

# Optimizers
optimizer_G = torch.optim.Adam(
    itertools.chain(Enc.parameters(), Dec.parameters()),
    lr=lr,
    betas=(b1, b2),
)
optimizer_D = torch.optim.Adam(D.parameters(), lr=lr, betas=(b1, b2))

# Learning rate update schedulers
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
    optimizer_G, lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step
)
lr_scheduler_D = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D, lr_lambda=LambdaLR(n_epochs, epoch, decay_epoch).step
)

reinforce_optimizer = torch.optim.SGD(StylePolicy.parameters(), lr=.0001, momentum=0.9, weight_decay=5e-4)

In [0]:
# Adversarial ground truths
valid = 1
fake = 0

log_file = open("/content/drive/My Drive/model/log.txt", 'a')

for epoch in trange(epoch, n_epochs, desc="Epoch"):
  torch.cuda.empty_cache()
  #save the initial weights for the REPTILE loop
  weights_original = deepcopy(StylePolicy.state_dict())
  new_weights = []
  #running_reward = 10

  for i in trange(batch_size, desc="Batch"):
    torch.cuda.empty_cache()
    #initialize empty tensors to save information to train the discriminators and generators
    content_code_stack = torch.empty(32, 2048, 2, 2)
    content_code_recovered_stack = torch.empty(32, 2048, 2, 2)
    style_code_stack = torch.empty(32, style_dim, 1, 1)
    style_code_recovered_stack = torch.empty(32, style_dim, 1, 1)
    X_stack = torch.empty(32, channels, img_height, img_width).cuda()

    #create random goal class one-hot encoded vector
    indices = torch.randint(0,10, size=(1,1))
    one_hot = torch.nn.functional.one_hot(indices, 10) # size=(4,7,n)
    goal = one_hot.squeeze(0).float().cuda()
    goal_reinforce = indices.long().cuda()[0]

    #generate gaussian noise image
    mu, sigma = 0, 1.0 # mean and standard deviation
    image = np.random.normal(mu, sigma, [img_height, img_width, channels])
    image = 255*(image - np.min(image))/np.ptp(image)
    X = image.astype(np.uint8)

    X = efficientnet_transform(transforms.ToPILImage()(X).convert("RGB"))
    X.unsqueeze_(0)
    X = Variable(X).to(device)

    efficientnet_features = efficientnet.extract_features(X)
    efficientnet_class = efficientnet.classify(efficientnet_features)
    policycode = StylePolicy.get_action(efficientnet_features, efficientnet_class, goal)
  
    #iterations of recurrent image development, 100 good enough? Potentially explore stacking samples for proper batching
    for reinforce_iteration in range(reinforce_iterations):
      X_focus = X[:, :, policycode[0]:policycode[0] + 16, policycode[1]:policycode[1] + 16]
      X_focus = torch.nn.functional.interpolate(X_focus, size=[img_height, img_width], mode = 'bilinear')
      
      content_code = efficientnet.extract_features(X_focus)
      style_code = torch.stack(policycode[2:]).unsqueeze(dim=0)

      # Reconstruct images
      X_hat = Dec(content_code, style_code)

      X[:, :, policycode[0]:policycode[0] + 16, policycode[1]:policycode[1] + 16] = \
                      torch.nn.functional.interpolate(X_hat, size=[16, 16], mode = 'bilinear')

      efficientnet_features = efficientnet.extract_features(X)
      Efficientnet_encoding = torch.nn.Softmax(dim=1)(efficientnet.classify(efficientnet_features))
      Efficientnet_encdoing = Efficientnet_encoding

      if reinforce_iteration == reinforce_iterations - 33:
        content_code_stack[reinforce_iteration, :, :, :] = content_code
        style_code_stack[reinforce_iteration, :, :, :] = style_code
        X_stack[reinforce_iteration, :, :, :] = X_hat

      #get the reward for each step taken during the inner model training for the REINFORCE algorithm
      reward = -1 * CrossEntropy_criterion(Efficientnet_encoding, goal_reinforce)

      StylePolicy.store_outcome(reward)

      policycode = StylePolicy.get_action(efficientnet_features, Efficientnet_encoding, goal)

      sys.stdout.write('\rEpoch %d Batch %d Iteration %d Reward %.2f%%' % (epoch, i, reinforce_iteration, reward))
      sys.stdout.flush()

      #save data if batch number is 0
      if i == 0:
        if not os.path.isdir("/content/drive/My Drive/model/images/{}".format(epoch)):
          os.makedirs("/content/drive/My Drive/model/images/{}".format(epoch))
        torchvision.utils.save_image(X, '/content/drive/My Drive/model/images/{}/iter_{}.jpg'.format(epoch, reinforce_iteration))
      line = "\r\rEpoch:{},Batch:{},Iter:{},Reward:{}".format(epoch, i, reinforce_iteration, reward)
      log_file.write(line)
      #print(line)

    # Cycle translation
    content_code_recovered_stack, style_code_recovered_stack = Enc(X_stack)

    # Losses
    loss_GAN = 5 * D.compute_loss(X_stack.detach(), valid)
    loss_s = 10 * L1_criterion(style_code_recovered_stack.cuda().detach(), style_code_stack.cuda().detach()) #.detach().item()
    loss_c = lambda_cont * L1_criterion(content_code_recovered_stack.cuda().detach(), content_code_stack.cuda().detach()) #.detach().item()
    
    optimizer_G.zero_grad()
    loss_G = loss_GAN + loss_s + loss_c
    loss_G.backward()
    optimizer_G.step()

    optimizer_D.zero_grad()
    true_image = next(iter(train_loader))[0]
    loss_D = D.compute_loss(true_image.cuda(), valid) + D.compute_loss(X_hat.detach(), fake)
    loss_D.backward()
    optimizer_D.step()

    #finish iteration - train REINFORCE algo
    reinforce_loss = StylePolicy.episode_finished()
    reinforce_optimizer.zero_grad()
    reinforce_loss.backward()
    reinforce_optimizer.step()
    log_file.write("Epoch:{},Batch:{},Reinforce_Loss:{}\n".format(epoch, i, reinforce_loss))

    #save trained weights & reload the weights from the original model
    new_weights.append(deepcopy(StylePolicy.state_dict()))
    StylePolicy.load_state_dict({ name: weights_original[name] for name in weights_original })

  #meta training step, REPTILE
  ws = len(new_weights)
  fweights = { name : new_weights[0][name]/float(ws) for name in new_weights[0] }
  for i in range(1, ws):
      #cur_weights = deepcopy(model.state_dict())
      for name in new_weights[i]:
          fweights[name] += new_weights[i][name]/float(ws)
  outerstepsize = outerstepsize0 * (1 - epoch / n_epochs) # linear schedule
  StylePolicy.load_state_dict({name : 
      weights_original[name] + ((fweights[name] - weights_original[name]) * outerstepsize) 
      for name in weights_original})
  
  # Update learning rates
  lr_scheduler_G.step()
  lr_scheduler_D.step()
  torch.save(StylePolicy.state_dict(), "/content/drive/My Drive/model/saved_models/%s/Policy.pth" % (dataset_name))
  torch.save(Enc.state_dict(), "/content/drive/My Drive/model/saved_models/%s/Enc.pth" % (dataset_name))
  torch.save(Dec.state_dict(), "/content/drive/My Drive/model/saved_models/%s/Dec.pth" % (dataset_name))
  torch.save(D.state_dict(), "/content/drive/My Drive/model/saved_models/%s/D.pth" % (dataset_name))


Batch:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 215 Batch 0 Iteration 0 Reward -2.43%

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Reinforce Loss : -0.005859375 Reinforce Max Reward : -2.375192165374756


Batch:  10%|█         | 1/10 [00:24<03:43, 24.87s/it]

...

Reinforce Loss : 0.00390625 Reinforce Max Reward : -2.4234485626220703


Batch:  90%|█████████ | 9/10 [02:34<00:16, 16.95s/it]

Reinforce Loss : 0.0087890625 Reinforce Max Reward : -2.4233551025390625


Batch:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 330 Batch 0 Iteration 2 Reward -2.44%

In [0]:
#del Policy
del Enc
del D
del Dec
torch.cuda.empty_cache()

In [0]:
test = next(iter(train_loader))[0]
test.shape

In [0]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [0]:
torch.cuda.empty_cache()